# Лабораторная работа №8

## Рекуррентные нейронные сети для анализа временных рядов

Набор данных для прогнозирования временных рядов, который состоит из среднемесячного числа пятен на солнце, наблюдаемых с января 1749 по август 2017.

Данные в виде csv-файла можно скачать на сайте _Kaggle_: https://www.kaggle.com/robervalt/sunspots/

### Задание 1

Загрузите данные. Изобразите ряд в виде графика. Вычислите основные характеристики временного ряда (сезонность, тренд, автокорреляцию).

### Задание 2

Для прогнозирования разделите временной ряд на обучающую, валидационную и контрольную выборки.

### Задание 3

Примените модель _ARIMA_ для прогнозирования значений данного временного ряда.

### Задание 4

Повторите эксперимент по прогнозированию, реализовав рекуррентную нейронную сеть (с как минимум 2 рекуррентными слоями).

### Задание 5

Сравните качество прогноза моделей.

Какой максимальный результат удалось получить на контрольной выборке?